# Import Libraries

In [8]:
import pandas as pd
import os
import glob
import pymongo
from pymongo import MongoClient
import csv
from dotenv import load_dotenv
import json

# Upload Data

In [5]:
df = pd.read_csv('/Users/investmentguy/Tableau_Bonfire_124/bonfire_124_streamlit_app/bonfire_124_streamlit_app/student-mat.csv', sep=";")
df1 = pd.read_csv('/Users/investmentguy/Tableau_Bonfire_124/bonfire_124_streamlit_app/bonfire_124_streamlit_app/student-por.csv', sep=";")

# Merge Both CSVs As One With Proper Columns

In [13]:
column_list = ["school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"]
df_final = df.merge(df1, on=column_list, how='outer')
df_final


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel_y,freetime_y,goout_y,Dalc_y,Walc_y,health_y,absences_y,G1_y,G2_y,G3_y
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4.0,3.0,4.0,1.0,1.0,3.0,4.0,0.0,11.0,11.0
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5.0,3.0,3.0,1.0,1.0,3.0,2.0,9.0,11.0,11.0
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4.0,3.0,2.0,2.0,3.0,3.0,6.0,12.0,13.0,12.0
3,GP,F,15,U,GT3,T,4,2,health,services,...,3.0,2.0,2.0,1.0,1.0,5.0,0.0,14.0,14.0,14.0
4,GP,F,16,U,GT3,T,3,3,other,other,...,4.0,3.0,2.0,1.0,2.0,5.0,0.0,11.0,13.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,MS,F,19,R,GT3,A,1,1,at_home,at_home,...,3.0,5.0,4.0,1.0,4.0,1.0,0.0,8.0,0.0,0.0
678,MS,F,18,R,GT3,T,2,2,services,other,...,4.0,2.0,1.0,1.0,1.0,4.0,5.0,14.0,14.0,15.0
679,MS,F,18,R,LE3,A,1,2,at_home,other,...,4.0,3.0,4.0,1.0,4.0,5.0,0.0,16.0,15.0,15.0
680,MS,F,19,R,GT3,T,1,1,at_home,other,...,4.0,3.0,3.0,1.0,1.0,3.0,4.0,7.0,8.0,9.0


# Clean The Data

* Normalize The Column Names
* Check Data Types
* Check Nulls
* Drop Columns?
* Cardinality?

In [20]:
# Normalize Column Names

df_final.columns = df_final.columns.str.replace(' ', '_').str.strip().str.lower()
df_final.head()

,school,sex,age,address,famsize,pstatus,medu,fedu,mjob,fjob,...,famrel_y,freetime_y,goout_y,dalc_y,walc_y,health_y,absences_y,g1_y,g2_y,g3_y
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4.0,3.0,4.0,1.0,1.0,3.0,4.0,0.0,11.0,11.0
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5.0,3.0,3.0,1.0,1.0,3.0,2.0,9.0,11.0,11.0
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4.0,3.0,2.0,2.0,3.0,3.0,6.0,12.0,13.0,12.0
3,GP,F,15,U,GT3,T,4,2,health,services,...,3.0,2.0,2.0,1.0,1.0,5.0,0.0,14.0,14.0,14.0
4,GP,F,16,U,GT3,T,3,3,other,other,...,4.0,3.0,2.0,1.0,2.0,5.0,0.0,11.0,13.0,13.0


# Checking Data Types

In [12]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 53 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   school        682 non-null    object 
 1   sex           682 non-null    object 
 2   age           682 non-null    int64  
 3   address       682 non-null    object 
 4   famsize       682 non-null    object 
 5   pstatus       682 non-null    object 
 6   medu          682 non-null    int64  
 7   fedu          682 non-null    int64  
 8   mjob          682 non-null    object 
 9   fjob          682 non-null    object 
 10  reason        682 non-null    object 
 11  guardian_x    407 non-null    object 
 12  traveltime_x  407 non-null    float64
 13  studytime_x   407 non-null    float64
 14  failures_x    407 non-null    float64
 15  schoolsup_x   407 non-null    object 
 16  famsup_x      407 non-null    object 
 17  paid_x        407 non-null    object 
 18  activities_x  407 non-null    

# Checking Nulls

In [19]:
df_final.isnull()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel_y,freetime_y,goout_y,Dalc_y,Walc_y,health_y,absences_y,G1_y,G2_y,G3_y
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
678,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
679,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
680,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [22]:
df_final[df_final['guardian_x'].isna() == True]

,school,sex,age,address,famsize,pstatus,medu,fedu,mjob,fjob,...,famrel_y,freetime_y,goout_y,dalc_y,walc_y,health_y,absences_y,g1_y,g2_y,g3_y
407,GP,M,16,U,GT3,T,2,3,other,other,...,3.0,2.0,3.0,2.0,2.0,1.0,4.0,13.0,12.0,13.0
408,GP,F,18,U,GT3,T,2,1,services,other,...,5.0,4.0,5.0,1.0,3.0,5.0,10.0,10.0,9.0,8.0
409,GP,F,17,U,LE3,A,2,1,other,other,...,3.0,2.0,2.0,1.0,2.0,5.0,8.0,11.0,10.0,11.0
410,GP,M,16,U,GT3,T,2,1,at_home,other,...,3.0,2.0,1.0,1.0,1.0,2.0,4.0,9.0,9.0,11.0
411,GP,F,16,U,GT3,A,2,2,other,other,...,5.0,3.0,4.0,1.0,1.0,5.0,12.0,13.0,11.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,MS,F,19,R,GT3,A,1,1,at_home,at_home,...,3.0,5.0,4.0,1.0,4.0,1.0,0.0,8.0,0.0,0.0
678,MS,F,18,R,GT3,T,2,2,services,other,...,4.0,2.0,1.0,1.0,1.0,4.0,5.0,14.0,14.0,15.0
679,MS,F,18,R,LE3,A,1,2,at_home,other,...,4.0,3.0,4.0,1.0,4.0,5.0,0.0,16.0,15.0,15.0
680,MS,F,19,R,GT3,T,1,1,at_home,other,...,4.0,3.0,3.0,1.0,1.0,3.0,4.0,7.0,8.0,9.0


In [16]:
# Checking out value types to determine if I want to replace the null values with certain string values since simply removing these rows will remove over 40% of my data

df_final['guardian_x'].value_counts()

guardian_x
mother    282
father     93
other      32
Name: count, dtype: int64

In [23]:
df_final[df_final['traveltime_x'].isnull() == True]

,school,sex,age,address,famsize,pstatus,medu,fedu,mjob,fjob,...,famrel_y,freetime_y,goout_y,dalc_y,walc_y,health_y,absences_y,g1_y,g2_y,g3_y
407,GP,M,16,U,GT3,T,2,3,other,other,...,3.0,2.0,3.0,2.0,2.0,1.0,4.0,13.0,12.0,13.0
408,GP,F,18,U,GT3,T,2,1,services,other,...,5.0,4.0,5.0,1.0,3.0,5.0,10.0,10.0,9.0,8.0
409,GP,F,17,U,LE3,A,2,1,other,other,...,3.0,2.0,2.0,1.0,2.0,5.0,8.0,11.0,10.0,11.0
410,GP,M,16,U,GT3,T,2,1,at_home,other,...,3.0,2.0,1.0,1.0,1.0,2.0,4.0,9.0,9.0,11.0
411,GP,F,16,U,GT3,A,2,2,other,other,...,5.0,3.0,4.0,1.0,1.0,5.0,12.0,13.0,11.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,MS,F,19,R,GT3,A,1,1,at_home,at_home,...,3.0,5.0,4.0,1.0,4.0,1.0,0.0,8.0,0.0,0.0
678,MS,F,18,R,GT3,T,2,2,services,other,...,4.0,2.0,1.0,1.0,1.0,4.0,5.0,14.0,14.0,15.0
679,MS,F,18,R,LE3,A,1,2,at_home,other,...,4.0,3.0,4.0,1.0,4.0,5.0,0.0,16.0,15.0,15.0
680,MS,F,19,R,GT3,T,1,1,at_home,other,...,4.0,3.0,3.0,1.0,1.0,3.0,4.0,7.0,8.0,9.0


In [24]:
df_final['traveltime_x'].value_counts()

traveltime_x
1.0    265
2.0    111
3.0     23
4.0      8
Name: count, dtype: int64

In [25]:
# Dropping all nulls due to the sake of time. Lost all of my notes and files permenantly last night and had to start over from scratch, so decided to take a break from the computer.

df_final.dropna(inplace=True)

In [26]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 382 entries, 0 to 405
Data columns (total 53 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   school        382 non-null    object 
 1   sex           382 non-null    object 
 2   age           382 non-null    int64  
 3   address       382 non-null    object 
 4   famsize       382 non-null    object 
 5   pstatus       382 non-null    object 
 6   medu          382 non-null    int64  
 7   fedu          382 non-null    int64  
 8   mjob          382 non-null    object 
 9   fjob          382 non-null    object 
 10  reason        382 non-null    object 
 11  guardian_x    382 non-null    object 
 12  traveltime_x  382 non-null    float64
 13  studytime_x   382 non-null    float64
 14  failures_x    382 non-null    float64
 15  schoolsup_x   382 non-null    object 
 16  famsup_x      382 non-null    object 
 17  paid_x        382 non-null    object 
 18  activities_x  382 non-null    objec

In [30]:
# Creating CSV to use to connect to MongoDB
df = df_final.to_csv('student_info.csv', index=False)

# Connecting to MongoDB

In [9]:
client = MongoClient('Mongo_URL')
db = client['bonfire_124_streamlit']
collection = db['ClusterO']

In [ ]:
reader = csv.DictReader(df)

for each in reader:
    row = {}
    for field in header:
        row[field] = each[field]

In [6]:
def connect_to_mongo(user=os.getenv('USERNAME'), password=os.getenv('PASSWORD')):
        load_dotenv()
        user = user
        password = password
        mongo_url = os.getenv('MONGO_URL')
        #Connect to PyMongo
        pymongo.MongoClient(mongo_url)
        
        
        
if __name__ == '__main__':
    c = connect_to_mongo()
    print('Successful Connection to Client Object')


Successful Connection to Client Object
